In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import datashader as ds
import holoviews as hv
import hvplot.xarray
import panel as pn
import geoviews as gv
import geoviews.feature as gf
from holoviews.operation.datashader import rasterize, shade
import cartopy.crs as ccrs
import cmocean
from pathlib import Path

In [ ]:
hv.extension('bokeh', 'matplotlib')

In [ ]:
# path = Path('~/Documents/cryo_geophysics/raw_data/BedMachineAntarctica-v3.nc')
path = Path(r'C:\Files\UF_school\cryo_geophysics\raw_data\BedMachineAntarctica-v3.nc')

bm = xr.load_dataset(path)
bm = bm.drop_vars(names='mapping')

In [ ]:
bm.bed.dims

In [ ]:
bm

In [ ]:
# bed_chunk = bm.bed.chunk(chunks={'x':1000,'y':1000})

In [ ]:
raster = rasterize(hv.Image(bm.bed)).opts(cmap=cmocean.cm.topo, clim=(-2500,2500), aspect='equal', 
                                 colorbar=True, frame_height=380, 
                                 title='BedMachine v3 - Bed', clabel='meters', tools=['hover'])

raster

In [ ]:
rangexy = hv.streams.RangeXY(source=raster)
rangexy

In [ ]:
raster.ddims

In [ ]:
rasterize(hv.Image(bm.thickness)).opts(cmap='plasma', clim=(0,3000), aspect='equal', 
                                 colorbar=True, frame_height=380, 
                                 title='BedMachine v3 - Thickness', clabel='meters', tools=['hover'])

In [ ]:
opts_vars = {
    'bed' : dict(cmap=cmocean.cm.topo, clim=(-2500,2500), clabel='meters'),
    'surface' : dict(cmap='viridis', clim=(float(bm.surface.min().values), float(bm.surface.max().values)), clabel='meters'),
    'thickness' : dict(cmap='plasma', clim=(float(bm.thickness.min().values), float(bm.thickness.max().values)), clabel='meters'),
    'firn' : dict(cmap='inferno', clim=(float(bm.firn.min().values), float(bm.firn.max().values)), clabel='meters'),
    'mask' : dict(cmap='viridis', clim=(float(bm.mask.min().values), float(bm.mask.max().values)), clabel='mask'),
    'errbed' : dict(cmap='viridis', clim=(float(bm.errbed.min().values), float(bm.errbed.max().values)), clabel='errbed'),
    'source' : dict(cmap='viridis', clim=(float(bm.source.min().values), float(bm.source.max().values)), clabel='source'),
    'dataid' : dict(cmap='viridis', clim=(float(bm.dataid.min().values), float(bm.dataid.max().values)), clabel='dataid'),
    'geoid' : dict(cmap='cividis', clim=(float(bm.geoid.min().values), float(bm.geoid.max().values)), clabel='meters')
}
for key in opts_vars:
    opts_vars[key].update(dict(colorbar=True, frame_height=380, aspect='equal', tools=['hover'], xlabel='PSX [m]', ylabel='PSY [m]'))

# opts_vars

In [ ]:
float(bm.mask.min().values)

In [ ]:
from holoviews.streams import RangeX, RangeY

def plot(var):
    var = var_select.value
    if var in ['bed', 'surface', 'thickness', 'geoid', 'firn']:
        raster = rasterize(hv.Image(bm[var]))
    else:
        raster = rasterize(hv.Image(bm[var]), aggregator='mode')
    return raster.opts(**opts_vars[var], title=f'BedMachine - {var}')

def on_var_select(event):
    var = event.obj.value
    col[-1] = plot(var=var)

var_select = pn.widgets.Select(name="value", options=list(bm.data_vars))
var_stream = hv.streams.Params(var_select, ['value'], rename={'value': 'var'})
var_select.param.watch(on_var_select, parameter_names=['value']);

col = pn.Column(var_select, plot(var_select.value))
col

In [ ]:
from holoviews.streams import RangeX, RangeY

var_select = pn.widgets.Select(name="value", options=list(bm.data_vars))

@pn.depends(var=var_select)
def update_plot(var):
    return plot(var)

col = pn.Column(var_select, update_plot)
col

In [ ]:
from holoviews.streams import RangeX, RangeY

def plot(var):
    image = hv.Image(bm[var], kdims=['x', 'y']).redim.label(x='PXS [m]', y='PSY [m]')
    image = image.opts(**opts_vars[var], title=f'BedMachine - {var}')
    return image.opts(backend_opts={"colorbar.title": opts_vars[var]['clabel']})

var_select = pn.widgets.Select(name="Field", options=list(bm.data_vars))

@pn.depends(var=var_select)
def update_plot(var):
    return plot(var)

@pn.depends(var=var_select)
def get_aggregator(var=var_select):
    if var in ['bed', 'surface', 'thickness', 'geoid', 'firn']:
        return 'mean'
    else:
        return 'mode'

dmap = rasterize(hv.DynamicMap(update_plot), aggregator=get_aggregator)

col = pn.Column(var_select, dmap)
col

In [ ]:
from holoviews.streams import RangeX, RangeY


def plot(var, cmap, clim):
    image = hv.Image(bm[var], kdims=['x', 'y']).redim.label(x='PXS [m]', y='PSY [m]')
    image = image.opts(**opts_vars[var], title=f'BedMachine - {var}')
    image = image.opts(cmap=cmap, clim=clim)
    return image.opts(backend_opts={"colorbar.title": opts_vars[var]['clabel']})

var_select = pn.widgets.Select(name="Field", options=list(bm.data_vars))
cmap_list = ['viridis', 'plasma', 'inferno', 'magma', 'cividis', 'cmo.topo', 'cmo.ice', 'cmo.deep', 'cmo.speed_r', 'cmo.rain_r', 
             'cmo.balance', 'cmo.curl', 'cmo.diff']
cmap_select = pn.widgets.Select(name='cmap', options=cmap_list)

#vmin_slider = pn.widgets.FloatSlider(name='Float Slider', start=0, end=3.141, step=0.01, value=1.57)

@pn.depends(var=var_select)
def get_clim_slider(var=var_select):
    start = float(bm[var].values.min())
    end = float(bm[var].values.max())
    step = (end-start)/100
    clim_slider = pn.widgets.RangeSlider(name='clim', start=start, end=end, step=step, value=(start, end))
    return clim_slider

clim_slider = get_clim_slider(var_select.value)

def reset_clim(var=var_select):
    start = float(bm[var].values.min())
    end = float(bm[var].values.max())
    step = (end-start)/100
    clim_slider.start = start
    clim_slider.end = end
    clim_slider.step = step
    clim_slider.value = (start, end)

@pn.depends(var=var_select, cmap=cmap_select, clim=clim_slider)
def update_plot(var=var_select, cmap=cmap_select, clim=clim_slider):
    reset_clim(var)
    return plot(var, cmap, clim)

@pn.depends(cmap=cmap_select, clim=clim_slider)
def update_cmap(cmap=cmap_select):
    return plot(var_select, cmap_select, clim_slider.value)

# @pn.depends(clim=clim_slider)
# def update_clim(clim=clim_slider):
#     return plot(var, cmap, clim_slider.value)

@pn.depends(var=var_select)
def get_aggregator(var=var_select):
    if var in ['bed', 'surface', 'thickness', 'geoid', 'firn']:
        return 'mean'
    else:
        return 'mode'



# @pn.depends(cmap=cmap_select)
# def get_cmap(cmap=cmap_select):
#     pass

dmap = rasterize(hv.DynamicMap(update_plot), aggregator=get_aggregator)
clim_slider_bind = pn.bind(update_cmap, clim_slider)
col = pn.Column(var_select, cmap_select, clim_slider, dmap)
col

In [ ]:
var_select

In [ ]:
class Mypanel:
    def __init__(self):
        self.var_select = pn.widgets.Select(name="Field", options=list(bm.data_vars))
        self.cmap_list = ['viridis', 'plasma', 'inferno', 'magma', 'cividis', 'cmo.topo', 'cmo.ice', 'cmo.deep', 'cmo.speed_r', 'cmo.rain_r', 
                     'cmo.balance', 'cmo.curl', 'cmo.diff']
        self.cmap_select = pn.widgets.Select(name='cmap', options=cmap_list)
    
        self.clim_slider = get_clim_slider(self.var_select.value)
    
        self.tmp_var = self.var_select.value

    def plot(self, var, cmap, clim):
        image = hv.Image(bm[var], kdims=['x', 'y']).redim.label(x='PXS [m]', y='PSY [m]')
        image = image.opts(**opts_vars[var], title=f'BedMachine - {var}')
        image = image.opts(cmap=cmap, clim=clim)
        return image.opts(backend_opts={"colorbar.title": opts_vars[var]['clabel']})

    def get_clim_slider(self):
        var=self.var_select
        start = float(bm[var].values.min())
        end = float(bm[var].values.max())
        step = (end-start)/100
        clim_slider = pn.widgets.RangeSlider(name='clim', start=start, end=end, step=step, value=(start, end))
        return clim_slider
    
    def reset_clim(self):
        var=self.var_select
        start = float(bm[var].values.min())
        end = float(bm[var].values.max())
        step = (end-start)/100
        self.clim_slider.start = start
        self.clim_slider.end = end
        self.clim_slider.step = step
        self.clim_slider.value = (start, end)
    
    @pn.depends(var=self.var_select, cmap=self.cmap_select, clim=self.clim_slider)
    def update_plot(self, var=var_select, cmap=cmap_select, clim=clim_slider):
        if self.tmp_var != var:
            #reset_clim(var)
            self.tmp_var = var.value
        return self.plot(var, cmap, clim)
    
    @pn.depends(cmap=self.cmap_select, clim=self.clim_slider)
    def update_cmap(self, cmap=cmap_select):
        return plot(var_select, cmap_select, clim_slider.value)
    
    # @pn.depends(clim=clim_slider)
    # def update_clim(clim=clim_slider):
    #     return plot(var, cmap, clim_slider.value)
    
    @pn.depends(var=var_select)
    def get_aggregator(self, var=var_select):
        if var in ['bed', 'surface', 'thickness', 'geoid', 'firn']:
            return 'mean'
        else:
            return 'mode'

In [ ]:
myp = Mypanel()

dmap = rasterize(hv.DynamicMap(myp.update_plot()), aggregator=myp.get_aggregator())
col = pn.Column(var_select, cmap_select, dmap)

In [ ]:
var_select.on_change

# Dashboard

https://examples.holoviz.org/gallery/datashader_dashboard/datashader_dashboard.html

In [ ]:
import os, colorcet, param as pm, holoviews as hv, panel as pn, datashader as ds
#import intake
from holoviews.element import tiles as hvts
from holoviews.operation.datashader import rasterize, shade, spread
from collections import OrderedDict as odict

In [ ]:
aggfns = odict([(f.capitalize(),getattr(ds,f)) for f in ['count','sum','min','max','mean','var','std']])
aggfns

In [ ]:
cmaps = odict([(x, plt.get_cmap(x)) for x in ['viridis', 'plasma', 'inferno', 'cividis', 'cmo.topo']])
cmaps

In [ ]:
aggfns = odict([(f,getattr(ds.reductions,f)) for f in ['count','sum','min','max','mean','var','std']])
aggfns

In [ ]:
fields = odict([(x, bm[x]) for x in bm.data_vars])
#aggfns = odict([(f,getattr(ds,f)) for f in ['count','sum','min','max','mean','var','std']])
aggfns = ['mean', 'mode']
cmaps = odict([(x, plt.get_cmap(x)) for x in ['viridis', 'plasma', 'inferno', 'cividis', 'cmo.topo']])

class Explorer(pm.Parameterized):
    field         = pm.Selector(objects=fields)
    agg_fn        = pm.Selector(objects=aggfns)
    cmap          = pm.Selector(objects=cmaps)
    
    @pm.depends('field')
    def elem(self):
        image = hv.Image(self.field, kdims=['x', 'y']).opts(**opts_vars[self.field.name], title=f'BedMachine - {self.field.name}')
        return image.redim.label(x='PXS [m]', y='PSY [m]')

    # @pm.depends('agg_fn')
    # def aggregator(self):
    #     field = None if self.field.name == "counts" else self.field
    #     return self.agg_fn(self.field.values)

    @pm.depends('agg_fn')
    def aggregator(self):
        if self.field in ['bed', 'surface', 'thickness', 'geoid', 'firn']:
            return 'mean'
        else:
            return 'mode'

    def viewable(self,**kwargs):
        rasterized = rasterize(hv.DynamicMap(self.elem), aggregator=self.aggregator, cmap=self.cmap, width=800, height=400)
        # shaded     = shade(rasterized, cmap=self.param.cmap)
        # spreaded   = spread(shaded, px=self.param.spreading, how="add")
        # dataplot   = spreaded.apply.opts(alpha=self.param.data_opacity, show_legend=False)
        
        return rasterized
    
explorer = Explorer(name="")

In [ ]:
fields = list(bm.data_vars)
#aggfns = odict([(f,getattr(ds,f)) for f in ['count','sum','min','max','mean','var','std']])
aggfns = ['mean', 'mode', 'var', 'std', 'min', 'max']
cmaps = ['viridis', 'plasma', 'inferno', 'cividis', 'cmo.topo']

class Explorer(pm.Parameterized):
    field         = pm.Selector(objects=fields)
    agg_fn        = pm.Selector(objects=aggfns)
    cmap          = pm.Selector(objects=cmaps)
    clim          = pm.Range(default=(0.,4.), bounds=(float(bm[fields[0]].min().values), float(bm[fields[0]].max().values)))
    diverging     = pm.Boolean(default=False)
    
    
    # @pm.depends('field')
    # def elem(self):
    #     image = hv.Image(bm[self.field], kdims=['x', 'y']).redim.label(x='PXS [m]', y='PSY [m]')
    #     image = image.opts(colorbar=True, frame_height=380, aspect='equal', tools=['hover'])
    #     return image

    @pm.depends('field', 'cmap', 'clim')
    def elem(self):
        image = hv.Image(bm[self.field], kdims=['x', 'y']).redim.label(x='PXS [m]', y='PSY [m]')
        image = image.opts(colorbar=True, frame_height=380, aspect='equal', tools=['hover'])
        # self.param.clim.bounds = ((float(bm[self.field].min().values), float(bm[self.field].max().values)))
        # self.clim = ((float(bm[self.field].min().values), float(bm[self.field].max().values)))
        image = image.opts(cmap=self.cmap, clim=self.clim, clabel=opts_vars[self.field]['clabel'])
        return image

    @pm.depends('field', watch=True)
    def update_clim(self):
        self.param.clim.bounds = ((float(bm[self.field].min().values), float(bm[self.field].max().values)))
        self.clim = ((float(bm[self.field].min().values), float(bm[self.field].max().values)))

    @pm.depends('diverging', 'clim', watch=True)
    def diverging_cmap(self):
        if self.diverging==True:
            vmin = np.min(np.abs([self.clim[0], self.clim[1]]))
            self.clim = (-vmin, vmin)

    @pm.depends('agg_fn')
    def aggregator(self):
        return self.agg_fn

    @pm.depends('cmap')
    def get_cmap(self):
        return self.cmap

    def viewable(self,**kwargs):
        rasterized = rasterize(hv.DynamicMap(self.elem), aggregator=self.aggregator)
        # shaded     = shade(rasterized, cmap=self.param.cmap)
        # shaded = shaded.opts(colorbar=True, frame_height=380, aspect='equal', tools=['hover'], xlabel='PSX [m]', ylabel='PSY [m]')
        # spreaded   = spread(shaded, px=self.param.spreading, how="add")
        # dataplot   = spreaded.apply.opts(alpha=self.param.data_opacity, show_legend=False)
        
        return rasterized
    
explorer = Explorer(name="")

In [ ]:
field         = pm.Selector(objects=fields)

In [ ]:
explorer.field.values

In [ ]:
explorer.agg_fn

In [ ]:
im = explorer.elem()
rasterize(im)

In [ ]:
explorer.agg_fn(explorer.field)

In [ ]:
panel = pn.Row(pn.Column(pn.Param(explorer.param, expand_button=False)), explorer.viewable())
panel.servable("Datashader Dashboard")

In [ ]:
opts_vars[explorer.field]['clabel']

In [ ]:
explorer.diverging

In [ ]:
hv.help

In [ ]:
bool(explorer.field.values)

In [ ]:
hv.help(shade)

In [ ]:
hv_ds = hv.Dataset(bm)
hv_ds

In [ ]:
bed = hv_ds.to(hv.Image)
bed

In [ ]:
# from xrviz.dashboard import Dashboard

# dash = Dashboard(bm)
# dash.panel()

In [ ]:
bm = bm.drop_vars(names='mapping')
bm

In [ ]:
ds = hv.Dataset(bm)
ds

In [ ]:
airtemp = ds.to(hv.Image, kdims=["x", "y"])
airtemp

In [ ]:
hv.help(rasterize)